In [199]:
import sympy
from sympy.abc import x,y,z
from cmath import pi

In [200]:
from ast import Return


class Rotations:
    def __init__(self):
        self.alfa  = sympy.Symbol('alfa')
        self.beta  = sympy.Symbol('beta')
        self.gamma = sympy.Symbol('gamma')
        self.theta = sympy.Symbol('theta')
        self.zRotM = sympy.Matrix([
            [sympy.cos(self.theta),-sympy.sin(self.theta), 0],
            [sympy.sin(self.theta), sympy.cos(self.theta), 0],
            [0                    , 0                    , 1]
        ])
        self.yRotM = sympy.Matrix([
            [sympy.cos(self.theta), 0,-sympy.sin(self.theta)],
            [0                    , 1, 0                    ],
            [sympy.sin(self.theta), 0, sympy.cos(self.theta)]
        ])
        self.xRotM = sympy.Matrix([
            [1, 0                    , 0                    ],
            [0, sympy.cos(self.theta),-sympy.sin(self.theta)],
            [0, sympy.sin(self.theta), sympy.cos(self.theta)]
        ])

    def zyxToRotM(self,z,y,x):
        return self.zRotM.subs(self.theta,z) * self.yRotM.subs(self.theta,y) * self.xRotM.subs(self.theta,x)

Rotations().zyxToRotM(pi/2,0,0)

Matrix([
[6.12323399573677e-17,                 -1.0, 0],
[                 1.0, 6.12323399573677e-17, 0],
[                   0,                    0, 1]])

In [201]:
class TwoLinkPlanar:

    def __init__(self, a1=None, a2=None) -> None:
        self.a1 = sympy.Symbol('a_1') if a1 is None else a1
        self.a2 = sympy.Symbol('a_2') if a2 is None else a2
        self.q1 = sympy.Symbol('q_1')
        self.q2 = sympy.Symbol('q_2')
        self.x  = x
        self.y  = y
        self.theta = sympy.Symbol('theta')
        self.direct  = lambda q1,q2: None
        self.inverse = lambda x,y: None
        self._resetSymbolsAndLambdas()

    def directKinSymbols(self):
        x_dk     = self.a1*sympy.cos(self.q1) + self.a2*sympy.cos(self.q1+self.q2)
        y_dk     = self.a1*sympy.sin(self.q1) + self.a2*sympy.sin(self.q1+self.q2)
        theta_dk = self.q1 + self.q2
        return sympy.Matrix((x_dk,y_dk,theta_dk))

    def inverseKinSymbols(self):
        q2_ik = sympy.acos((self.x**2 + self.y**2 - self.a1**2 - self.a2**2)/(2*self.a1*self.a2))
        q1_ik = sympy.atan2(self.y,self.x) - sympy.atan2(self.a2*sympy.sin(q2_ik), self.a1 + self.a2*sympy.cos(q2_ik))
        return sympy.Matrix((q1_ik,q2_ik))

    def setLengths(self,a1,a2):
        self.a1 = a1
        self.a2 = a2
        self._resetSymbolsAndLambdas()
    
    def setJointCoordinates(self,joint1,joint2):
        self.q1 = joint1
        self.q2 = joint2
        self._resetSymbolsAndLambdas()

    def setCartesianCoordinates(self,coord1,coord2):
        self.x = coord1
        self.y = coord2
        self._resetSymbolsAndLambdas()
    
    def _resetSymbolsAndLambdas(self):
        # Compute simbolic kinematics
        self.directSym  = self.directKinSymbols()
        self.inverseSym = self.inverseKinSymbols()
        # Compute lambda kinematic functions 
        try:
            self.direct  = sympy.lambdify((self.q1,self.q2),self.directSym)
            self.inverse = sympy.lambdify((self.x,self.y),self.inverseSym)
        except SyntaxError:
            self.direct  = lambda q1,q2: None
            self.inverse = lambda x,y: None

In [202]:
twp = TwoLinkPlanar(2,3)
twp.directKinSymbols()

Matrix([
[2*cos(q_1) + 3*cos(q_1 + q_2)],
[2*sin(q_1) + 3*sin(q_1 + q_2)],
[                    q_1 + q_2]])

In [203]:
twp.direct(pi/2,pi/2)

array([[-3.        ],
       [ 2.        ],
       [ 3.14159265]])

In [204]:
twp.inverse(-1,2)

array([[0.46364761],
       [2.30052398]])

In [217]:
class TwoLinkAndBase():

    def __init__(self, a1=None, a2=None) -> None:
        self.a1 = sympy.Symbol('a_1') if a1 is None else a1
        self.a2 = sympy.Symbol('a_2') if a2 is None else a2
        self.q1 = sympy.Symbol('q_1')
        self.q2 = sympy.Symbol('q_2')
        self.q3 = sympy.Symbol('q_3')
        self.x = x
        self.y = y
        self.z = z
        self._r = sympy.sqrt(self.x**2 + self.y**2)
        self.twoLink = TwoLinkPlanar(a1,a2)
        self.twoLink.setCartesianCoordinates(self._r, self.z)
        self.twoLink.setJointCoordinates(self.q2,self.q3)
        # Initialize symbolic kinematics
        self.directSym  = self.directKinSymbols()
        self.inverseSym = self.inverseKinSymbols()   
        # Initialize lambdas
        self.direct  = lambda q1,q2,q3: None
        self.inverse = lambda x, y, z : None
        # Cmpute symbolic kinematics and lambda functions
        self._resetSymbolsAndLambdas()

    def directKinSymbols(self):
        r_dk,z_dk,pitch_dk = self.twoLink.directKinSymbols()
        x_dk = r_dk*sympy.cos(self.q1)
        y_dk = r_dk*sympy.sin(self.q1)
        yaw_dk = self.q1
        roll_dk = 0
        return sympy.Matrix((x_dk,y_dk,z_dk,yaw_dk,pitch_dk,roll_dk))

    def directKinHomoTSymbols(self):
        x_dk,y_dk,z_dk,yaw_dk,pitch_dk,roll_dk = self.directKinSymbols()
        rotM = Rotations().zyxToRotM(yaw_dk,pitch_dk,roll_dk)
        translation = sympy.Matrix((x_dk,y_dk,z_dk))
        return sympy.Matrix((
            (rotM            , translation    ),
            (sympy.zeros(1,3), sympy.ones(1,1))
        ))

    def inverseKinSymbols(self):
        q1_ik = sympy.atan2(self.y,self.x)
        q2_ik,q3_ik = self.twoLink.inverseKinSymbols()
        return sympy.Matrix((q1_ik,q2_ik,q3_ik))
        
    def _resetSymbolsAndLambdas(self):
        # Compute simbolic kinematics
        self.directSym  = self.directKinSymbols()
        self.inverseSym = self.inverseKinSymbols()
        # Compute lambda kinematic functions 
        try:
            self.direct  = sympy.lambdify((self.q1,self.q2,self.q3),self.directSym)
            self.inverse = sympy.lambdify((self.x, self.y, self.z), self.inverseSym)
        except SyntaxError:
            self.direct  = lambda q1,q2,q3: None
            self.inverse = lambda x, y, z : None

tlb = TwoLinkAndBase(4,3)
tlb.directKinHomoTSymbols()

Matrix([
[cos(q_1)*cos(q_2 + q_3), -sin(q_1), -sin(q_2 + q_3)*cos(q_1), (4*cos(q_2) + 3*cos(q_2 + q_3))*cos(q_1)],
[sin(q_1)*cos(q_2 + q_3),  cos(q_1), -sin(q_1)*sin(q_2 + q_3), (4*cos(q_2) + 3*cos(q_2 + q_3))*sin(q_1)],
[         sin(q_2 + q_3),         0,           cos(q_2 + q_3),            4*sin(q_2) + 3*sin(q_2 + q_3)],
[                      0,         0,                        0,                                        1]])

In [211]:
tlb.inverseKinSymbols()

Matrix([
[                                                                                                              atan2(y, x)],
[atan2(z, sqrt(x**2 + y**2)) - atan2(3*sqrt(1 - (x**2/24 + y**2/24 + z**2/24 - 25/24)**2), x**2/8 + y**2/8 + z**2/8 + 7/8)],
[                                                                                acos(x**2/24 + y**2/24 + z**2/24 - 25/24)]])

In [212]:
tlb.direct(0,pi/4,pi/4)

array([[2.82842712],
       [0.        ],
       [5.82842712],
       [0.        ],
       [1.57079633],
       [0.        ]])

In [213]:
tlb.inverse(4,0,4)

array([[0.        ],
       [0.25338261],
       [1.27482752]])